In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle
import csv

# Config Path 설정

In [2]:
import yaml

with open('./config-video.yaml') as f:
    conf = yaml.full_load(f)
    

In [3]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
global cuda_dev
cuda_dev = conf['trn_args']['device_id']

device = torch.device(cuda_dev if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:',device)

True
학습을 진행하는 기기: cuda:3


### [option]dataset 확인 (형태확인 및 피쳐수 확인 필요시)

In [4]:
with open(conf['dataset']['chat_path'],"rb") as f1:  
    chat=pickle.load(f1)

with open(conf['dataset']['audio_path'],"rb") as f2:  
    audio=pickle.load(f2)
with open(conf['dataset']['video_path'],"rb") as f2:  
    video=pickle.load(f2)
with open(conf['dataset']['label_path'],"rb") as f4:  
    real_result=pickle.load(f4)

In [5]:
len(video['102844412722519367'])
len(real_result)

112

# Dataset

video 형태가 안맞았는지, 혼자만 코드 이상하게 되어있음. 

기본 형태 : 'gameid':[[0초 피쳐],[1초 피쳐]]

video는 1개의 피쳐가 그냥 'gameid':[0초피쳐,1초피쳐,,,,]로 되어있음.
 --> 추후 수정시, 

**s_window+=list(self.video[game_id][vframe+idx])**

로만 바꿔주시오~


In [4]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type
        self.gt_range =  1-conf['trn_args']['hl_range']
        with open(conf['dataset']['chat_path'],"rb") as f1:  
            self.chat=pickle.load(f1)
       
        with open(conf['dataset']['audio_path'],"rb") as f2:  
            self.audio=pickle.load(f2)
        with open(conf['dataset']['video_path'],"rb") as f2:  
            self.video=pickle.load(f2)
        with open(conf['dataset']['label_path'],"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
        if d_type =='total':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654'] + ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630'] + ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
    
        
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.real_result[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)
        
        begin_pos = 0 
        hl_frames = []
        for it, cur_pos in enumerate(pos_idx):
            if it+1 < len(pos_idx): 
                if((pos_idx[it+1] - cur_pos) > 1):#cur_pos와 cur_pos+1 간격이 1보다 크면, 즉 다른 구간이면
                    begin = int((it+1 - begin_pos) * self.gt_range) + begin_pos
                    hl_frames.extend( pos_idx[begin: it] ) #한구간의 하이라이트 25%만 사용하겠다.
                    begin_pos = it+1
        
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[hl_frames] = len(sampling) / float(len(hl_frames))
        self.WeightedSampling = sampling
        
        self.sum=np.insert(np.cumsum([len(self.audio[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return self.sum[-1]
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            window_size=conf['trn_args']['window_size']
            std_size = min(len(self.audio[game_id]),min(len(self.chat[game_id]),len(self.video[game_id]))) #데이터가 가장 짧은거에 맞춤 (오류방지)
            chat_size = conf['dataset']['chat_size']#c초당 피쳐 개수
            audio_size = conf['dataset']['audio_size']
            video_size = conf['dataset']['video_size']
            global total_size
            total_size = video_size
            
            #video_only
            
            for idx in range(window_size): #7 : window size
                s_window=[]
                if vframe+idx<std_size:#아래는 데이터 형태가 조금씩 달라서 다음과 같이 진행.
                    #s_window+=list((self.chat[game_id][vframe+idx]))#vframe의 chat
                    #s_window+=list(self.audio[game_id][vframe+idx])#vframe의 image
                    s_window+=[self.video[game_id][vframe+idx]]
                else:
                    #s_window=[0,0,0]#padding value
                    s_window=[0]*(total_size)#초가 초과되는 경우 0으로 피쳐 패
                window.append(s_window)


            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [5]:
train=Mul_data('train')
val=Mul_data('val')

#test
print("\n\ndatset testing....")
print(train[100])
print(len(train[100][1]))
print("\n****************\n")


#loader 
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=conf['trn_args']['sampling'])
train_loader=torch.utils.data.DataLoader(train,batch_size=conf['trn_args']['trn_bs'],sampler=sampler1)
# train_loader=torch.utils.data.DataLoader(train,batch_size=32)
val_loader=torch.utils.data.DataLoader(val,batch_size=conf['trn_args']['val_bs'])

data load fin
data load fin


datset testing....
('102844412722519367', array([[0.01117067],
       [0.02793789],
       [0.03156882],
       [0.03493765],
       [0.03568352],
       [0.03391392],
       [0.03102885]]), 0)
7

****************



# Modeling

In [6]:

class LSTM(nn.Module):
    def __init__(self):
        
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,num_layers,batch_first=True)
        self._lin = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.to(device)
        hidden = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).to(device) # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).to(device) # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        out,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        feature = out[:,-1,:]
        out = self._lin(out[:,-1,:])
        return out,feature
        

# Experiment 

### measures

In [7]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [8]:
##best 값을 확인하기위함

def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

### experiment

In [9]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    if epoch<20:
        lr = 0.01
    else:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr

In [10]:
hyper_parameter = [128,256]
weight_directory=['_d128/','_d256/',]




for exp_index in range(0,1):

    global weight_dir
    weight_dir= conf['dataset']['weight_path']+weight_directory[exp_index]##weight 저장
    model_epoch = conf['trn_args']['epoch']

    global input_size,hidden_size,num_layers
    input_size=total_size
    hidden_size=hyper_parameter[exp_index]
    num_layers=conf['trn_args']['num_layer']


    lr = conf['trn_args']['lr']

    model=LSTM().to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr,momentum=0.9,weight_decay=1e-4)


    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    with open(weight_dir+'train_result','a') as f:
        f.write('=====result=======\n')
    f1_best=0
    for epoch in range(model_epoch):
        lr = adjust_learning_rate(optimizer,epoch)
        #lr=0.01
        losses = AverageMeter()
        top1 = AverageMeter()
        print(epoch)
        model.train()
        for i, (g,inputs,labels) in enumerate(train_loader):
            inputs=inputs.float()
            inputs=inputs.to(device)
            labels=labels.to(device)
            optimizer.zero_grad()
            out,_=model(inputs)
            out=out.to(device)
            loss=criterion(out,labels)
            losses.update(loss,labels.size(0))
            loss.backward()
            optimizer.step()

        model.eval()
        val_losses=AverageMeter()
        acc=0
        gt_sum=0
        tp_sum=0
        fp_sum=0
        fn_sum=0
        acc=0
        sum=0
        pred_sum=0


        with open(weight_dir+'train_result','a') as f:

            with torch.no_grad():
                for it, (g,inputs,labels) in enumerate(val_loader):
                    inputs=inputs.float()
                    inputs=inputs.to(device)
                    labels=labels.to(device)
                    out,_=model(inputs)
                    out=out.to(device)
                    loss=criterion(out,labels)
                    val_losses.update(loss,labels.size(0))
                    TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(out.cpu(),labels.cpu())
                    tp_sum += TP
                    fp_sum += FP
                    fn_sum += FN
                    pred_sum += pred_len
                    gt_sum += gt_len
                    acc=acc+TP+TN
                    sum+=len(out)
                if tp_sum>0 and fp_sum>0 and fn_sum>0:
                    precision = tp_sum/(tp_sum+fp_sum)
                    recall = tp_sum / (tp_sum+fn_sum)
                    f1 = (2*precision*recall / (precision + recall))
                    accuracy=acc/sum
                    print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    csv_f = open(weight_dir+'result.csv','a', newline='')
                    wr = csv.writer(csv_f)
                    wr.writerow([epoch,precision, recall,f1])
                    csv_f.close()
                    torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))
                    if f1_best<f1:
                        f.write("== best epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                        torch.save(model.state_dict(),'{}'.format(weight_dir+"best"))
                        f1_best=f1

                else:
                    print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                    f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                    csv_f = open(weight_dir+'result.csv','a', newline='')
                    wr = csv.writer(csv_f)
                    wr.writerow([epoch,0, 0,0])
                    csv_f.close()
                    torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))                




    def fmeasure2(frames,label): ##measure def for test
        average = [0,0,0,0,0]
        for key in frames.keys():
            TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
            FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
            TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
            FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
            precision = TP/(TP+FP)
            recall = TP/(TP+FN)
            accuracy = (TP+TN)/(TP+FN+FP+TN)
            if precision==0 and recall == 0:
                print('!')
            else:
                f1 = (2*precision*recall / (precision + recall))
                print(key)
                print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
                average[0]+= precision
                average[1]+= recall
                average[2]+= f1
                average[3]+= accuracy
                average[4]+=1
        print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))
        with open(weight_dir+'/eval','a') as f:
            f.writelines('prec:{}, recall:{}, f1:{}, acc : {}\n'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))
            f.close()
    test=Mul_data('test')
    test_loader=torch.utils.data.DataLoader(test,batch_size=conf['trn_args']['val_bs'])
    dataset=weight_dir+'best'
    checkpoint=torch.load(dataset,map_location=device)
    model.load_state_dict(checkpoint)
    model.eval()
    pred_sum = 0#model output
    gt_sum = 0#label
    tp_sum=0
    fp_sum=0
    fn_sum=0
    acc=0
    sum=0
    result={}


    #evaluation
    with torch.no_grad():
        for it, (game_id,inputs,labels) in enumerate(test_loader):
            inputs=inputs.float()
            labels=labels
            output,_=model(inputs)
            TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
            for idx,g in enumerate(game_id):
                if g not in result.keys():
                    result[g]=pred[idx].tolist()
                else:
                    result[g]+=pred[idx].tolist()
    with open(conf['dataset']['label_path'],"rb") as f4:  
        real_result=pickle.load(f4)
    fmeasure2(result,real_result)

    #test



    test=Mul_data('total')
    test_loader=torch.utils.data.DataLoader(test,batch_size=1)
    dataset=weight_dir+'best'
    checkpoint=torch.load(dataset,map_location=device)
    model.load_state_dict(checkpoint)
    model.to(device)
    model.eval()

    result={}
    with torch.no_grad():
        for it, (game_id,inputs,labels) in enumerate(test_loader):
            inputs=inputs.float()
            labels=labels
            output_p,output_f=model(inputs)

            if game_id[0] not in result.keys():
                print(game_id[0])
                result[game_id[0]]=[(output_f[0]).tolist()]

            else:
                result[game_id[0]]+=[(output_f[0]).tolist()]



    with open(weight_dir+'video_feature.json','a') as f:
        json.dump(result,f)

0
epoch 0 train_loss : 0.6934036016464233 , val_loss : 0.6756197214126587,p 0, r 0, f 0

1
epoch 1 train_loss : 0.6933951377868652 , val_loss : 0.6786527633666992,p 0, r 0, f 0

2
epoch 2 train_loss : 0.693248450756073 , val_loss : 0.7280842661857605,p 0, r 0, f 0

3
epoch 3 train_loss : 0.6932936310768127 , val_loss : 0.722899854183197,p 0, r 0, f 0

4
epoch 4 train_loss : 0.6932012438774109 , val_loss : 0.6772322058677673,p 0, r 0, f 0

5
epoch 5 train_loss : 0.6931425929069519 , val_loss : 0.6966245174407959,p 0, r 0, f 0

6
epoch 6 train_loss : 0.6930710673332214 , val_loss : 0.7032290101051331,p 0, r 0, f 0

7
epoch 7 train_loss : 0.6928499341011047 , val_loss : 0.7037250995635986,p 0, r 0, f 0

8
epoch 8 train_loss : 0.6929491758346558 , val_loss : 0.6776668429374695,p 0, r 0, f 0

9
epoch 9 train_loss : 0.6924803256988525 , val_loss : 0.6769144535064697,p 0, r 0, f 0

10
epoch 10 train_loss : 0.6920995712280273 , val_loss : 0.6950551867485046,p 0, r 0, f 0

11
epoch 11 train_los

epoch 62 train_loss : 0.4334307909011841 , val_loss : 0.4274427592754364,p 0.6550970873786408, r 0.5958057395143488, f 0.6240462427745664

63
epoch 63 train_loss : 0.430972158908844 , val_loss : 0.4352642297744751,p 0.6287686996547756, r 0.6030905077262693, f 0.6156619718309859

64
epoch 64 train_loss : 0.4279451370239258 , val_loss : 0.42786240577697754,p 0.6429249762583096, r 0.5977924944812362, f 0.6195378631892016

65
epoch 65 train_loss : 0.4288884699344635 , val_loss : 0.42541223764419556,p 0.6505298651252408, r 0.5962472406181015, f 0.622206864777701

66
epoch 66 train_loss : 0.4294385015964508 , val_loss : 0.4372098743915558,p 0.6262580054894785, r 0.6044150110375276, f 0.6151426645697596

67
epoch 67 train_loss : 0.4317854642868042 , val_loss : 0.42777660489082336,p 0.6555582583313062, r 0.5949227373068433, f 0.6237703969447981

68
epoch 68 train_loss : 0.4307897090911865 , val_loss : 0.43764522671699524,p 0.6141328577797592, r 0.608167770419426, f 0.6111357586512867

69
epoch

102844341905011343
102844235753356742
102844235750997440
102844412709346612
102844412705217835
102844235752963525
102844412712164667
102844412705545516
102844341912220311
102844341907370644
102844235749424575
102844212429419722
102844294669568199
102844212431779031
102844294666422466
102844224146472059
102844212428895431
102844212429747404
102844235748703677
102844224146930812
102844212430730450
102844294674876621
102844341909598870
102844283020453971
102844294670026952
102844412723174729
102844341904683662
102844283025696858
102844235747261881
102844401154168486
102844235748310460
102844412711836986
102844412723567946
102844235749031358
102844294674286796
102844294666881219
102844412716686654
102844294671796427
102844224145685626
102844412717407552
102844235751390657
102844401156069033
102904869420860038
102910307641576395
102844341905404560
102844341906977427
102844212430075086
102844412711116088
102844401153578660
102844294667405508
102844412706659630
102844212431058132
102844341902

#

In [8]:
with open(weight_dir+'video_feature.json','r') as f:
  temp = json.load(f)

In [12]:
for i in temp.keys():
    print(len(temp[i]))
    break

2069
